In [91]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import json
import os

In [92]:
# Abre o arquivo JSON
# scrap_path = 'scraped/2024-04-19/wouerner_brand_o/___red_flag_que_pouc'
scrap_path = 'last_scrap/'
with open(scrap_path + 'data.json', 'r', encoding="UTF-8") as f:
    # Carrega o conteúdo do arquivo JSON em um dicionário Python
    data = json.load(f )

# Agora 'data' contém o conteúdo do arquivo JSON como um dicionário Python
data


{'autor_name': 'Abraham Lincoln',
 'autor_subtitle': 'UI Designer | UX Designer | Product Designer | Designer de Interface do Usuário | Designer de Experiência do Usuário | Figma',
 'post_age': '6d',
 'autor_img_src': 'https://static.licdn.com/aero-v1/sc/h/9c8pery4andzj6ohjkjp54ma2',
 'content_text': 'Fala pessoal, tudo bem?\n\nEstou um tempinho sem postar, mas venho com novidades 🙂\n\nGostaria de compartilhar o projeto que participei na SouJunior\n\nÉ um local muito legal onde sempre tem algo para aprender, os mentores são super disponíveis e sempre ajudar os juninhos 🐣\n\nEsse projeto é sobre a dashboard que ajudei a desenvolver como ui/ux designer, não é a versão final, mas ajudou o time a ter uma visão do que a SouJunior precisava e fico feliz de saber que está em andamento.\n\nGostaria e agradecer ao Wouerner Brandão pela iniciativa da SouJunior, a Vanêssa Santana por sempre estar por perto para acompanhar o andamento, ao Vinicius L Miguel por sempre ajudar a todos e ser um excele

In [93]:
def remove_emoji(text):
    text_clean = ""
    for caractere in text:
        if not ord(caractere) > 0xFFFF:
            text_clean += caractere
    return text_clean


def break_line(texto, line_max=75):
    text_splited = texto.split("\n")

    final_text = []
    for i, line in enumerate(text_splited):
        if len(line) > line_max:
            words = line.split()
            line_text = words[0]

            # links
            if len(line_text) > line_max:
                line_text = line_text[: line_max - 3] + "..."

            cursor = len(words[0])
            for word in words[1:]:
                if cursor + len(word) + 1 > line_max:
                    final_text.append(line_text)
                    line_text = ""
                    cursor = 0
                else:
                    line_text += " "
                    cursor += 1
                line_text += word
                cursor += len(word)

            final_text.append(line_text)

        else:
            final_text.append(line)

    final_text_str = "\n".join(final_text)
    return final_text_str

In [94]:
data["content_text"] += "\nawdasda"
data["content_text"] += "\nLoren Loren"
data["content_text"] += "\n aAAAAAxczcswacAAAAAAAAAAAAAALoren"
data["content_text"] += "\nLoren Loren"

In [95]:
def create_circle_mask(size):
    mask = Image.new("L", size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + size, fill=255)
    return mask


def write_text(
    draw,
    text_,
    pos=(),
    font_="segoeui",
    font_size=24,
    color=(0, 0, 0),
    multline=False,
    spacing=5,
):
    font = ImageFont.truetype(f"fonts/{font_}.ttf", font_size)
    if not multline:
        draw.text(pos, text_, font=font, fill=color)
    else:
        draw.multiline_text(pos, text_, font=font, fill=color, spacing=spacing)


def paste_image(image, path, pos, size=(), rounded=False):
    new_image = Image.open(path)

    if not size:
        size = (new_image.width, new_image.height)

    if rounded:
        mask = create_circle_mask(new_image.size)
        new_image.putalpha(mask)

    new_image = new_image.resize(size)

    image.paste(new_image, pos, mask=new_image)


def place_autor_header(image, draw, data, content_top_y):
    autor_name = data["autor_name"]
    autor_subtitle = (
        data["autor_subtitle"][:65] + "..."
        if len(data["autor_subtitle"]) > 65
        else data["autor_subtitle"]
    )
    post_time_stamp = data["post_age"]

    autor_image_path = scrap_path + "autor_img.png"
    paste_image(
        image, autor_image_path, pos=(135, content_top_y), size=(75, 75), rounded=True
    )

    write_text(draw, autor_name, (225, content_top_y), font_="seguisb")
    write_text(
        draw,
        autor_subtitle,
        (225, content_top_y + 30),
        font_size=20,
        color=(130, 130, 130),
    )

    write_text(
        draw,
        post_time_stamp,
        (225,
        content_top_y + 55),
        font_size=20,
        color=(130, 130, 130),
    )


def build_post_content_image(data, text, output_count, continued, end, height):
    if output_count > 10:
        print("???")
        return

    image = Image.open("backgrounds/3c.png")
    draw = ImageDraw.Draw(image)

    rectangle_size = (860, height)
    rectangle_pos = (
        int((image.width - rectangle_size[0]) / 2),
        int((image.height - rectangle_size[1]) / 2),
    )

    paste_image(
        image, "backgrounds/rectangle.png", size=rectangle_size, pos=rectangle_pos
    )
    content_top_y = rectangle_pos[1] + 20

    place_autor_header(image, draw, data, content_top_y)

    if continued:
        paste_image(
            image,
            "backgrounds/header_ellipsis.png",
            pos=(
                int((image.width - 800) / 2),
                rectangle_pos[1] + 110,
            ),
        )

    content_text_padding_top = (
        content_top_y + 97 if output_count == 1 else content_top_y + 165
    )

    # content text
    write_text(
        draw,
        text,
        (136, content_text_padding_top),
        multline=True,
        font_size=22,
        font_="seguiemj",
        spacing=12
    )

    # footer
    if end:
        paste_image(
            image,
            "backgrounds/action_bar.png",
            size=(int(800 * 1), int(65 * 1)),
            pos=(
                int((image.width - 800) / 2),
                rectangle_pos[1] + rectangle_size[1] - 80,
            ),
        )
        

        
    else:
        paste_image(
            image,
            "backgrounds/action_bar_ellipsis_continue.png",
            pos=(
                int((image.width - 800) / 2),
                rectangle_pos[1] + rectangle_size[1] - 72,
            ),
        )


    image.save(f"image_output/square_post_{output_count}.png")

In [96]:
def build_post_images(data, content_text, output_count=1):
    max_lines_per_imgage = 24
    line_height = 29

    text = break_line(content_text)

    text_splited = text.split("\n")
    n_lines = len(text_splited)
    print(n_lines)

    if n_lines > max_lines_per_imgage:
        text = "\n".join(text_splited[:max_lines_per_imgage])

        page_text = "\n".join(text_splited[max_lines_per_imgage:])
        height = (250 if output_count == 1 else 270) + n_lines * line_height
        build_post_images(data, page_text, output_count + 1)
        full = False

    # remover o else, se passou pela recursividade, vai chegar tratado no ultimo e tratar cada separado abaixo
    # else:
    #     print("Criando imagem com x linhas:", len(text.split('\n')))
    #     print("Output", output_count)
    #     print("End: True")
    #     # build_post_content_image(text= text, output_count=output_count, full= True, height=height)

    print("Criando imagem com x linhas:", len(text.split("\n")))
    print("Output", output_count)
    print("Continued:", output_count > 1)
    print("End:", n_lines < max_lines_per_imgage)


    text_to_build = "\n".join(text_splited[:max_lines_per_imgage])
    height = (200 if output_count == 1 else 280) + len(text.split("\n")) * line_height
    build_post_content_image(
        data,
        text=text_to_build,
        output_count=output_count,
        continued=output_count > 1,
        end=n_lines < max_lines_per_imgage,
        height = height
    )


build_post_images(data, data["content_text"])

29
5
Criando imagem com x linhas: 5
Output 2
Continued: True
End: True
Criando imagem com x linhas: 24
Output 1
Continued: False
End: False


In [97]:
new_data = data.copy()

In [98]:
# data = new_data.copy()
# for comment in data["comments"]:
#     comment["comment_text"] += "\n Lorem Ipsum Lorem Ipsum Lorem Ipsum"

data["comments"][0]["comment_text"] += "\n Lorem Ipsum Lorem Ipsum Lorem Ipsum"
data["comments"][0]["comment_text"] += "\n a"

In [99]:


def build_comments_image(data_comments, rectangle_height, output_count=1):

    image = Image.open("backgrounds/3c.png")
    draw = ImageDraw.Draw(image)

    # rectangle
    rectangle_height_base = rectangle_height  # if output_count == 1 else 330
    rectangle_size = (860, rectangle_height_base)
    rectangle_pos = (
        int((image.width - rectangle_size[0]) / 2),
        int((image.height - rectangle_size[1]) / 2),
    )

    # rectangle
    paste_image(
        image, "backgrounds/rectangle.png", size=rectangle_size, pos=rectangle_pos
    )

    # ellipsis continued
    paste_image(
        image,
        "backgrounds/header_ellipsis.png",
        pos=(
            int((image.width - 800) / 2),
            rectangle_pos[1] + 10,
        ),
    )


    comment_bg = "backgrounds/comment_bg.png"
    # comment 0

    comment_top_y = rectangle_pos[1] + 80

    for comment in data_comments:

        comment_autor = remove_emoji(comment["autor"])
        comment_subtitle = (
            comment["subtitle"][:65] + "..."
            if len(comment["subtitle"]) > 65
            else comment["subtitle"]
        )
        comment_age = comment["age"]
        comment_img_path = f"last_scrap/comment_profile_photo_{comment['image_index']}.png"
        comment_text = break_line(remove_emoji(comment["comment_text"]), line_max=65)

        comment_lines = len(comment_text.split("\n"))
        background_size = 100 + int(30 * comment_lines)


        
        #bg
        paste_image(
            image,
            comment_bg,
            pos=(215, comment_top_y),
            size=(720, background_size)
            
        )

        # paste image autor
        paste_image(
            image,
            comment_img_path,
            pos=(135, comment_top_y),
            size=(65, 65),
            rounded=True,
        )

        # name
        write_text(
            draw, comment_autor, (230, comment_top_y + 10), font_="seguisb", font_size=22
        )

        # headline
        write_text(
            draw,
            comment_subtitle,
            (230,
            comment_top_y + 40),
            font_size=20,
            color=(130, 130, 130),
            font_= "segoeuil"
        )

        # age
        write_text(
            draw,
            comment_age,
            (880,
            comment_top_y + 10),
            font_size=20,
            color=(130, 130, 130),
        )

        # text
        write_text(
            draw,
            comment_text,
            (230,
            comment_top_y + 80),
            font_size=22,
            font_="seguiemj",
            multline=True,
            spacing= 15
        )

        comment_top_y += background_size + 30

    image.save(f"image_output/comments_{output_count}.png")


def paginate_comments(comments, output_count = 1):
    n_comments = len(comments)
    print(n_comments)

    max_height = 950 #900
    base_height = 90
    height = 110
    staged_comments = []


    for i, comment in enumerate(comments):
        comment_text = break_line(remove_emoji(comment["comment_text"]), line_max=65)
        n_lines = len(comment_text.split("\n"))
        if n_lines > 23:
            continue
        comment_height = n_lines * 30 + base_height + 30

        if height + comment_height > max_height:
            paginate_comments(comments[i:], output_count=output_count + 1)
            break
        else:
            height+= comment_height
            staged_comments.append(comment)
    
    build_comments_image(staged_comments, height, output_count= output_count)
    print(height)
    return 1

paginate_comments(data["comments"])

5
2
410
890


1